## 1.3 ADC,TDC信号处理
### 目的：学习处理ADC、TDC记录的信号的处理方法
#### 实验设置：中子飞行时间谱测量（参照1.1描述）
### 物理过程
- 束流轰击靶, 以一定几率产生带电粒子，n，gamma等粒子。假设n的产生率之和为Rn；gamma的比例为Rg。n,gamma都可以被中子探测器探测(pid=0/1:gamma/neutron)。重带电粒子碎片被靶后Dipole magnet偏走，而部分轻粒子没有被完全偏走，仍然进入中子探测器，这部分占束流的比例为Rc(pid=2,下面代码中假设为质子)。其余的事件，即比例为1-Rn-Rg-Rc的事件在探测器上不产生信号(pid=3)。

### Trigger
- DAQ的触发由靶前塑料闪烁体探测器给出，即靶上有粒子入射时产生trigger。trigger经过适当的处理后成为ADC的Gate和TDC的start信号。

### ADC & TDC

- 常用ADC的输入信号幅度范围为0-4V, AD转换后，记录为0-4095(12-bit插件)的整数, 其数值称为道值(channel)，每道对应幅度为$LSB=4000/4095 \approx 1 mV$。当输入信号幅度超过最大范围时(称为超界)，ADC将其记录为最高道值4095(取决于ADC的具体数据格式，有些12-bit ADC记为8191道)。


- TDC记录stop信号($t_{stop}$)和start信号($t_{start}$)的差值$\Delta t=t_{stop}-t_{start}$。对于12位TDC，当量程为 $\Delta T$ =200 ns时,$LSB=200/4095 \approx 50$ ps。 当$\Delta t >\Delta T$,或无stop信号(超界)，记录为4095道(取决于TDC的具体数据格式，有些12-bit TDC记为8191道)。


- ADC在Gate的宽度范围内寻找最大幅度（如下图所示），将其转换成道值。当Gate的范围内没有实际信号时，ADC记录Gate宽度内基线噪声的最大值。这些事件累积起来画成直方图(histogram)的话, 将形成以基线平均幅度为中心，有一定展宽的峰，此峰位和$\sigma$值暂记为$V_{noise}，\sigma_{noise}$，峰的宽度取决于基线中叠加的噪声大小。常用的ADC一般只能记录输入信号的正幅度。当输入信号基线电平为负值(如-50mV)时, 为避免小信号(如脉冲幅度<50mV)不能被正常记录，在ADC设置一个正电平(由用户设置，如$V_{ADCbase}=200道$)，此时ADC记录的信号基线平均道值为$V_{noise}+V_{ADCbase}$，称为pedestal值，简称pedal值。因此ADC记录到的脉冲幅度=ADC道值-pedal。一般取$pedal+3 \sigma_{noise}$作为噪声和正常信号的界限。


- 一般在获取中为了减少插件到后端计算机之间传输数据量(数据传输时间是DAQ死时间的主要来源)，ADC将道值低于$pedal+3 \sigma_{noise}$的事件丢掉。上述步骤称之为zero-supression。类似地，TDC在传输前将超界信号丢弃，此步骤称为 overflow supression.

#### ADC/QDC Gate_Signal & Pedestal peak
![1](adc_ped.png)

- 实验中通过调节每个探测器的放大倍数、信号的时间延迟，将感兴趣的信号调到ADC和TDC的量程范围之内，这里gamma和中子事件是我们感兴趣的事件。其他不感兴趣的信号，如轻带电粒子信号则不要求一定在量程范围内。


-  <font color=red>TDC的超界信号为无效信号；ADC的低于噪声阈值之下的信号为无效信号，ADC的超界信号虽然幅度不确定，但仍具有一定意义(即为某种大信号)。</font>

### 代码说明
<font color=blue>通过调节电子学，使中子和gamma的大部分事件的time、energy落在TDC，ADC的量程范围内；pid=3的事件在ADC的Gata范围内无实际物理信号(pedal)，在TDC的量程范围内没有stop信号。</font>


``` cpp

// 将下列代码保存到treeADC.cc
// 在ROOT环境中运行：
// .x treeADC.cc
// 生成 treeADC.root 文件
void treeADC()
{
  const Double_t D=500.;//cm, distance between target and the scin.(Center)
  const Double_t L=100.;//cm, half length of the scin.
  const Double_t dD=5.;//cm, thickness of the scin.
  const Double_t TRes=1.;//ns, time resolution(FWHM) of the scintillator.
  const Double_t Lambda=380.;//cm, attenuation lenght of the scin.
  const Double_t QRes=0.1;//relative energy resolution(FWHM) of the scin. 
  const Double_t Vsc=7.5;//ns/cm, speed of light in the scin.
 //neutron & gamma
  const Double_t En0=100;//MeV, average neutron energy
  const Double_t EnRes=50.;//MeV, energy spread of neutron(FWHM)
  const Double_t Eg0=10;//MeV, gamma energy  

  const Double_t Rn=0.75;//ratio of neutron 
  const Double_t Rg=0.05;//ratio of gamma; 

 //charged particle hited on Neutron Detector, 
 const Double_t Rc=0.1;//ratio of proton
 const Double_t Ec0=50;//MeV
 const Double_t EcRes=50;//MeV energy spread of proton(FWHM)

 //ADC
 const Double_t ADCgain=60;//1MeV=60ch.
 const Double_t ADCuPed=100;//baseline of ADC for upper side
 const Double_t ADCdPed=120;//                for bottom side
 const Double_t ADCnoise=10;//sigma of noise
 const Int_t    ADCoverflow=4095;

 //TDC
 const Double_t TDCch2ns=40.;//1ns=40ch.
 const Int_t TDCoverflow=4095;
  
  TFile *opf=new TFile("treeADC.root","recreate");//新文件tree.root的指针 *opf
  TTree *opt=new TTree("tree","tree structure");//新tree的指针 *opt
    
  // 定义tree的branch变量
  Double_t x;
  Double_t e;
  int pid;    //0/1/2/3: gamma/neutron/proton/No_particle
  Double_t tof, ctof;
  Double_t tu, td;
  Double_t qu, qd;
  Int_t itu,itd;//TDC
  Int_t iqu,iqd;//ADC

  Double_t tu_off=5.5;//time offset
  Double_t td_off=20.4;//time offset
 
  // 将变量分支添加到tree结构中,第一个参数为变量名称，第二个为上面定义的变量地址，第三个为变量的类型说明，D表示Double_t。
  opt->Branch("x", &x, "x/D");//position
  opt->Branch("e", &e, "e/D");//energy
  opt->Branch("tof", &tof, "tof/D");//time of flight
  opt->Branch("ctof",&ctof,"ctof/D");//TOF from exp. data
  opt->Branch("pid", &pid, "pid/I");
  // raw time and energy
  opt->Branch("tu", &tu, "tu/D");
  opt->Branch("td", &td, "td/D");
  opt->Branch("qu", &qu, "qu/D");
  opt->Branch("qd", &qd, "qd/D");
  
  // energy in ADC, time in TDC 注意，以下Branch变量声明的类型为 Integer，
  opt->Branch("itu", &itu, "itu/I");
  opt->Branch("itd", &itd, "itd/i");
  opt->Branch("iqu", &iqu, "iqu/i");
  opt->Branch("iqd", &iqd, "iqd/i");

  
  TRandom3 *gr=new TRandom3(0);
  // 循环，逐事件往tree结构里添加对应分支信息。
  for(int i=0;i<1000000;i++){
    x=gr->Uniform(-L, L);
    Double_t Dr=D+gr->Uniform(-0.5,0.5)*dD;
    Double_t d=TMath::Sqrt(Dr*Dr+x*x);//cm, flight path
    //pid
    Double_t ratio=gr->Uniform();
    if(ratio<Rg) pid=0;
    if(ratio>=Rg && ratio<Rg+Rn) pid=1;
    if(ratio>=Rg+Rn && ratio<Rg+Rn+Rc) pid=2;
    if(ratio>=Rg+Rn+Rc) pid=3;
    
    // energy & tof    
    if(pid==0) {//gamma
       e=Eg0;
       tof=3*(d*0.01);
      }
    if(pid==1) { //neutron
       e=gr->Gaus(En0, EnRes/2.35); // neutron
       tof=72./TMath::Sqrt(e)*(d*0.01);//ns
    }
    if(pid==2) {//proton
      e=gr->Gaus(Ec0, EcRes/2.35); // proton
      tof=72./TMath::Sqrt(e)*(d*0.01);
    }
    if(pid==3) {
      e=-1;
      tof=-1;
    }
    
    //tu,td,qu,qd
    if(pid==3) {//
      tu=TDCoverflow;
      td=TDCoverflow;
      qu=ADCuPed+gr->Gaus(0,ADCnoise);
      qd=ADCdPed+gr->Gaus(0,ADCnoise);
    }
    else {
      //time
      tu=tof+(L-x)/Vsc+gr->Gaus(0,TRes/2.35)+tu_off;
      td=tof+(L+x)/Vsc+gr->Gaus(0,TRes/2.35)+td_off;
      tu *=TDCch2ns;
      td *=TDCch2ns;
      
      //energy depositon in the plastic
      Double_t q0;
      if(pid!=2) q0=e*ADCgain*gr->Uniform();//neutron &gamma,
      else q0=e*ADCgain; //charged particle
      
      //resolution
      q0=gr->Gaus(q0,q0*QRes/2.35);
      
      //light transmission within the plastic
      qu=q0*TMath::Exp(-(L-x)/Lambda);
      qd=q0*TMath::Exp(-(L+x)/Lambda);
      
      //threshold of time of tu and td
      if(qu<100) tu=TDCoverflow;
      if(qd<100) td=TDCoverflow;

      //ADC
      qu += gr->Gaus(ADCuPed,ADCnoise);
      qd += gr->Gaus(ADCdPed,ADCnoise);
      if(qu<0) qu=0;//no negative values
      if(qd<0) qd=0;
    }

      //overflow check
    if(tu>TDCoverflow) tu=TDCoverflow;
    if(td>TDCoverflow) td=TDCoverflow;
    if(qu>ADCoverflow) qu=ADCoverflow;
    if(qd>ADCoverflow) qd=ADCoverflow;
    
  //digitization
    itu=Int_t(tu);
    itd=Int_t(td);
    iqu=Int_t(qu);
    iqd=Int_t(qd);
 
    
    opt->Fill();
   
  }
  // 将数据写入root文件中
  opt->Write();
  opf->Close();
  
}

```

In [1]:
%jsroot on

In [2]:
TFile *ipf=new TFile("treeADC.root");//root -l tree.root

In [3]:
TCanvas *c1=new TCanvas();//* 在ROOT环境下可省略
c1->Clear();//* 在ROOT环境下可省略

### TDC 分析

In [4]:
//将多个histogram叠加到一张图上显示的方法-I，用Draw(“same”)选项， y轴的大小取决于第一个画出的图。
tree->Draw("itu>>htu(420,0,4200)");
TH1D *htu=(TH1D*)gROOT->FindObject("htu");//得到histogram的指针，通过指针进行进一步操作。
tree->Draw("itd>>htd(420,0,4200)");
TH1D *htd=(TH1D*)gROOT->FindObject("htd");
htd->SetLineColor(kGreen);
htu->Draw();
htd->Draw("same");
c1->SetLogy();
c1->Draw();

In [5]:
//将多个histogram叠加到一张图上显示的方法-II, 自动调整每张图的y轴显示范围。
THStack *hs = new THStack("hs","test stacked histograms");
hs->Add(htu);
hs->Add(htd);
hs->Draw("nostack");//替换成hs->Draw();语句，观察效果。
c1->Draw();

In [6]:
//二维关联图
tree->Draw("itu:itd>>(420,0,4200,420,0,4200)","","colz");//加入pid条件观察图的变化，结合实验设置理解图中不同区域的含义。
gPad->SetLogz();
gPad->SetLogy(0);
c1->Draw();//* 在ROOT环境下可省略

#### TDC条件的运用

In [7]:
//位置分布
tree->Draw("itu-itd");
gPad->SetLogy();
c1->Draw();//结果是否正常？如果异常，问题出在哪里？

In [8]:
tree->Draw("itu-itd>>hix(3000,-2000,1000)","itu<4095 && itd<4095");//只有在合理的取值范围的前提下，才能对参数进行运算
gPad->SetLogy(0);
c1->Draw();

### TDC 的显示问题

In [9]:
tree->Draw("itu-itd>>(2500,-2000,1000)","itu<4095 && itd<4095");//TDC 道值
gPad->SetLogy(0);
c1->Draw();//显示异常

In [10]:
tree->Draw("tu-td>>(2500,-2000,1000)","itu<4095 && itd<4095");//原始参数
gPad->SetLogy(0);
c1->Draw();

In [11]:
tree->Draw("itu*0.3>>(1200,0,1200)");
c1->Draw();

In [12]:
tree->Draw("tu*0.3>>(1200,0,1200)");
c1->Draw();

### 思考题
对比上面5个图，你有什么领悟？

### QDC分析
- 计算ped峰参数，确定噪声阈值

In [13]:
tree->Draw("iqu>>hquall(420,0,4200)");
TH1D *hquall=(TH1D*)gROOT->FindObject("hquall");
hquall->SetLineColor(kBlack);

tree->Draw("iqu>>hqu1(420,0,4200)","pid==0 || pid==1");
TH1D *hqu1=(TH1D*)gROOT->FindObject("hqu1");
hqu1->SetLineColor(kRed);

tree->Draw("iqu>>hqu2(420,0,4200)","pid==2");
TH1D *hqu2=(TH1D*)gROOT->FindObject("hqu2");
hqu2->SetLineColor(kGreen);


tree->Draw("iqu>>hqu3(420,0,4200)","pid==3");
TH1D *hqu3=(TH1D*)gROOT->FindObject("hqu3");
hqu3->SetLineColor(kBlue);
THStack *hsqu=new THStack("hsqu","qu with different pid");

In [14]:
hsqu->Add(hquall);
hsqu->Add(hqu1);
hsqu->Add(hqu2);
hsqu->Add(hqu3);
hsqu->Draw("nostack");
gPad->SetLogy();
c1->Draw();//辨认pedal峰，超界位置。

In [15]:
tree->Draw("iqd>>hqdall(420,0,4200)");
TH1D *hqdall=(TH1D*)gROOT->FindObject("hqdall");
hqdall->SetLineColor(kBlack);

tree->Draw("iqd>>hqd1(420,0,4200)","pid==0 || pid==1");
TH1D *hqd1=(TH1D*)gROOT->FindObject("hqd1");
hqd1->SetLineColor(kRed);

tree->Draw("iqd>>hqd2(420,0,4200)","pid==2");
TH1D *hqd2=(TH1D*)gROOT->FindObject("hqd2");
hqd2->SetLineColor(kGreen);


tree->Draw("iqd>>hqd3(420,0,4200)","pid==3");
TH1D *hqd3=(TH1D*)gROOT->FindObject("hqd3");
hqu3->SetLineColor(kBlue);
THStack *hsqd=new THStack("hsqd","qd with different pid");

In [16]:
hsqd->Add(hqdall);
hsqd->Add(hqd1);
hsqd->Add(hqd2);
hsqd->Add(hqd3);
hsqd->Draw("nostack");
gPad->SetLogy();
c1->Draw();//辨认pedal峰，超界位置

#### 拟合ped峰，提取拟合参数

In [17]:
hquall->Fit("gaus","","",0,250);
hqdall->Fit("gaus","","",0,250);
c1->Draw();

 FCN=313.042 FROM MIGRAD    STATUS=CONVERGED      73 CALLS          74 TOTAL
                     EDM=1.54199e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.51191e+04   4.43892e+01   2.98693e-01  -1.32088e-07
   2  Mean         1.41495e+02   1.29005e-01   1.08893e-03   9.92411e-04
   3  Sigma        5.16929e+01   1.09473e-01   4.82873e-06   1.25900e-01
 FCN=821.747 FROM MIGRAD    STATUS=CONVERGED      59 CALLS          60 TOTAL
                     EDM=1.52504e-11    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.49495e+04   4.44193e+01   4.76030e-01   3.14351e-09
   2  Mean         1.31740e+02   1.28989e-01   1.78137e-03   1.20836e-05
   3  Sigma        5.25816e+01   

In [18]:
TF1 *fgaus[2];
Double_t ped[2],sigma[2];//u,d
TString sq[2]={"qu","qd"};

In [19]:
fgaus[0]=hquall->GetFunction("gaus");
fgaus[1]=hqdall->GetFunction("gaus");
for(int i=0;i<2;i++) {
    ped[i]=fgaus[i]->GetParameter(1);
    sigma[i]=fgaus[i]->GetParameter(2);
    //TString的格式化输出。用法与printf一致。复杂格式输出推荐用TString::Form()。
    TString ss;
    ss.Form("ped_%s=%.2f, sigma_%s=%.2f",sq[i].Data(),ped[i],sq[i].Data(),sigma[i]); 
    cout<<ss<<endl;
}

ped_qu=141.50, sigma_qu=51.69
ped_qd=131.74, sigma_qd=52.58


In [20]:
tree->Draw("iqu:iqd>>(420,0,4200,420,0,4200)","","colz");
c1->SetLogz();
c1->SetLogy(0);
c1->Draw();//* 在ROOT环境下可省略

### ROOT命令的“ ”内加入动态数据的方法-TString::Form()

In [21]:
//qu,qd减去ped，去掉噪声及超界值，选择合理的取值范围
TString stree,scut;
stree.Form("(iqu-%f):(iqd-%f)>>hsud(200,0,4000,200,0,4000)",ped[0],ped[1]);//在""内加入动态数据的方法
scut.Form("iqu>(%f+3*%f) && iqu<4000 && iqd>(%f+3*%f) && iqd<4000",ped[0],sigma[0],ped[1],sigma[1]);
tree->Draw(stree.Data(),scut.Data(),"colz");
c1->Draw();
cout<<"tree: "<<stree.Data()<<endl;
cout<<"cut: "<<scut.Data()<<endl;

tree: (iqu-141.495044):(iqd-131.740239)>>hsud(200,0,4000,200,0,4000)
cut: iqu>(141.495044+3*51.692905) && iqu<4000 && iqd>(131.740239+3*52.581573) && iqd<4000


### 设置参数的别名：tree->SetAlias()

In [22]:
TString squa,sqda;
squa.Form("iqu-%f",ped[0]);
sqda.Form("iqd-%f",ped[1]);
tree->SetAlias("qua",squa.Data());
tree->SetAlias("qda",sqda.Data());
TCut cpid="pid=2";
tree->Draw("sqrt(qua*qda)",scut.Data());
gPad->SetLogy();
c1->Draw();

In [23]:
TString stcut="itu>0&&itu<4000&&itd>0&&itd<4000";
scut=scut+" && "+stcut;
tree->Draw("itd-itu:log(qua/qda)",scut.Data(),"colz");
c1->SetLogz();
c1->SetLogy(0);
c1->Draw();